In [ ]:
# default_exp benchmarking

In [ ]:
#export
from ms_empire.visualizations import *

In [ ]:
#export
import pandas as pd
def get_tps_fps(result_df, prot2org_file, thresh = 0.05, fc_thresh = 0.3):
    annotated = annotate_dataframe(result_df, prot2org_file)
    display(annotated)
    condpairs = result_df["condpair"].drop_duplicates()


    for condpair in condpairs:
        annotated_condpair = annotated[annotated["condpair"]==condpair]
        num_tps = sum(annotated_condpair["TP"])
        num_fps = sum(annotated_condpair["FP"])
        annotated_fcfilt = annotated_condpair[annotated["fc"] >fc_thresh]
        num_regulated_prots = sum(annotated_fcfilt["fdr"]<thresh)
        num_true_positives = sum(annotated_fcfilt["TP"] &(annotated_fcfilt["fdr"]<0.05))
        num_false_positives = sum(annotated_fcfilt["FP"] &(annotated_fcfilt["fdr"]<0.05))

        print(f'condpair {condpair}')
        print(f"total TPs {num_tps}")
        print(f"total FPs {num_fps}")
        print(f'regulated {num_regulated_prots}')
        print(f'false positives {num_false_positives}')
        print(f'true positives {num_true_positives}')
        print(f'regulated control {num_false_positives+num_true_positives}')
        print(f'precision {num_false_positives/num_regulated_prots}')
        print(f'precision {num_false_positives/num_regulated_prots}')


def annotate_dataframe(result_df, prot2org_file):
    prot2org = pd.read_csv(prot2org_file, sep = "\t")
    prot2org["FP"] = (prot2org["organism"] == "Homo sapiens")
    prot2org["TP"] = (prot2org["organism"] == "Saccharomyces cerevisiae")
    prot2org = prot2org[(prot2org["FP"] | prot2org["TP"])]
    print(f"df size before {len(result_df.index)}")
    annotated = pd.merge(result_df, prot2org, how='inner', on = "protein")
    print(f"df size after {len(annotated.index)}")
    return annotated

In [ ]:
#export
import pandas as pd
import matplotlib.pyplot as plt
def compare_to_reference(result_df, reference_file, condpair):#put in condpair as tuple
    result_df = result_df[result_df["condpair"]==condpair]

    ref_df = pd.read_csv(reference_file, sep = "\t")
    merged = pd.merge(result_df, ref_df, how='inner', on = "protein",suffixes = ["", "_ref"])
    display(merged)
    ax_p = merged.plot.scatter(x='pval_ref',y='pval')
    plt.show()
    ax_fc = merged.plot.scatter(x='log2FC_ref',y='fc')
    plt.show()
    ax_fdr = merged.plot.scatter(x='fdr_ref',y='fdr')
    plt.show()

In [ ]:
#export
import pandas as pd
import matplotlib.pyplot as plt
def compare_normalization(ref_normalization_file, norm1_df, norm2_df):
    ref_normed = pd.read_csv(ref_normalization_file, sep ="\t").set_index('peptide')
    display(ref_normed)

    merged = pd.merge(norm1_df, norm2_df, how='inner',  left_index = True, right_index = True)
    columns = merged.columns
    merged = pd.merge(ref_normed, merged, how='inner', left_index = True, right_index = True, suffixes = ["_ref", ""])
    display(merged)
    for i in range(len(columns)):
        sample1 = columns[i]
        sample2 = sample1+"_ref"
        ax_p = merged.plot.scatter(x=sample1,y=sample2)
        plt.show()

In [ ]:
#export
import pandas as pd
import numpy as np

def compare_to_reference(peptide_detail_file, result_df, peptide_df, protref_file, outdir):
    protein_ref = pd.read_csv(peptide_detail_file, sep="\t", usecols=["protein", "protein_pval", "protein_fc"]).drop_duplicates().rename(columns = {"protein_pval" : "pval_ref", "protein_fc": "fc_ref"})
    peptide_ref = pd.read_csv(peptide_detail_file, sep='\t', usecols = ["peptide", "protein", "peptide_pval","peptide_fc"]).rename(columns = {"peptide_pval" :"peptide_pval_ref", "peptide_fc" : "peptide_fc_ref"})
    compare_peptid_protein_overlaps(protein_ref, result_df, peptide_ref, peptide_df)
    compare_significant_proteins(result_df, protref_file)

    print_nonref_hits(protein_ref, result_df, peptide_ref, peptide_df, outdir)
    prots_merged = pd.merge(protein_ref, result_df, on = "protein", how='inner')
    peps_per_prot_ref = pd.DataFrame(peptide_ref.groupby(by=["protein"])['peptide'].count()).rename(columns = {"peptide":"num_peptides_ref"}).reset_index()
    prots_merged = pd.merge(prots_merged, peps_per_prot_ref, on = "protein", how='inner')

    peptides_merged = pd.merge(peptide_ref, peptide_df, on = "peptide", how='inner')

    peptides_merged["peptide_pval_diff"] = ( peptides_merged["peptide_pval"]/peptides_merged["peptide_pval_ref"]).abs()
    peptides_merged = peptides_merged.sort_values(by=['peptide_pval_diff'], ascending = False)
    display(peptides_merged.head(10))
    peptides_merged.to_csv(f"{outdir}/merged_peptides.tsv", sep = "\t", index = False)
    scatter_df_columns(prots_merged)
    scatter_df_columns(peptides_merged)

    prots_merged["pvaldiff"] = (np.log2(prots_merged["pval"]) - np.log2(prots_merged["pval_ref"])).abs()
    prots_merged = prots_merged.sort_values(by=['pvaldiff'], ascending = False)
    peptides_merged.to_csv(f"{outdir}/merged_proteins.tsv", sep = "\t", index = False)
    display(prots_merged.head(10))
    display(peptides_merged)


    prots_merged["numpep_diff"] = (prots_merged["num_peptides"] - prots_merged["num_peptides_ref"]).abs()
    prots_merged = prots_merged.sort_values(by=['numpep_diff'], ascending = False)
    display(prots_merged.head(10))

In [ ]:
#export
from matplotlib_venn import venn2
from matplotlib import pyplot as plt

def compare_significant_proteins(result_df, protref_file):
    protein_ref = pd.read_csv(protref_file, sep="\t")
    sigprots_ref = protein_ref[protein_ref["fdr"]<0.05]
    sigprots = result_df[result_df["fdr"]<0.05]
    prots_ref = set(sigprots_ref["protein"].to_list())
    prots = set(sigprots["protein"].to_list())
    print(f"in ref only {prots_ref - prots}")
    print(f"in AP only {prots-prots_ref}")
    venn2([prots_ref, prots], ('sigprots_ref', 'sigprots'))
    plt.show()

In [ ]:
#export
def print_nonref_hits(protein_ref, protein_df, peptide_ref, peptide_df, outdir):
    prots_nonref_df =  protein_df[~(protein_df["protein"].isin(protein_ref["protein"].to_list()))]#the tilde inverts the boolean vector
    peps_nonref_df = peptide_df[~(peptide_df["peptide"].isin(peptide_ref["peptide"].to_list()))]
    prots_nonref_df.to_csv(f"{outdir}/nonref_proteins.tsv", sep = "\t", index = False)
    peps_nonref_df.to_csv(f"{outdir}/nonref_peptides.tsv", sep = "\t", index = False)
    display(peps_nonref_df)